<a href="https://colab.research.google.com/github/netra-poonia/simulation1/blob/main/Relationship_Manager_Bonus_Calculator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### `Base Code`

In [1]:
import csv
from collections import defaultdict
import datetime

In [2]:
# vintage BAU
VOLUME_PAYOUT_SLABS = {
    'secured': [
        {'threshold_multiplier': 0, 'bonus_rate': 0.000},
        {'threshold_multiplier': 30, 'bonus_rate': 0.0020},
        {'threshold_multiplier': 50, 'bonus_rate': 0.0025},
        {'threshold_multiplier': 80, 'bonus_rate': 0.004},
        {'threshold_multiplier': 120, 'bonus_rate': 0.005},
    ],
    'secured_direct': [
        {'threshold_multiplier': 0, 'bonus_rate': 0.0012},
        {'threshold_multiplier': 10, 'bonus_rate': 0.0018},
        {'threshold_multiplier': 30, 'bonus_rate': 0.0025},
        {'threshold_multiplier': 60, 'bonus_rate': 0.0030},
    ],
    'unsecured': [
        {'threshold_multiplier': 0, 'bonus_rate': 0.000},
        {'threshold_multiplier': 5, 'bonus_rate': 0.004},
        {'threshold_multiplier': 15, 'bonus_rate': 0.007},
        {'threshold_multiplier': 30, 'bonus_rate': 0.01},
    ],
}

BAU_SECURED_EFFORT_MATRIX = {
    (0, 0):            {(0, 0): 1000, (1, 1): 1200, (2, 2): 1500, (3, 3): 1800, (4, float('inf')): 2000},
    (1, 4):            {(0, 0): 1200, (1, 1): 1500, (2, 2): 1800, (3, 3): 2000, (4, float('inf')): 3000},
    (5, 6):            {(0, 0): 1500, (1, 1): 1800, (2, 2): 3000, (3, 3): 3500, (4, float('inf')): 4000},
    (7, float('inf')): {(0, 0): 2000, (1, 1): 2500, (2, 2): 3500, (3, 3): 4000, (4, float('inf')): 5000},
}

BAU_UNSECURED_EFFORT_MATRIX = {
    (0, 0):                {(0, 0): 0,   (1, 3): 300, (3, 4): 400, (5, 7): 500, (8, float('inf')): 750},
    (1, 9):                {(0, 0): 300, (1, 3): 400, (3, 4): 500, (5, 7): 750, (8, float('inf')): 850},
    (10, 13):              {(0, 0): 400, (1, 3): 500, (3, 4): 750, (5, 7): 850, (8, float('inf')): 1000},
    (14, float('inf')):    {(0, 0): 500, (1, 3): 750, (3, 4): 850, (5, 7): 1000, (8, float('inf')): 1200},
}

# vintage 0-3 Months
VINTAGE_VOLUME_SLABS_0_3M = {
    'secured': [
        {'threshold_multiplier': 0, 'bonus_rate': 0.000},
        {'threshold_multiplier': 5, 'bonus_rate': 0.0025},
        {'threshold_multiplier': 20, 'bonus_rate': 0.0035},
        {'threshold_multiplier': 30, 'bonus_rate': 0.0045},
        {'threshold_multiplier': 45, 'bonus_rate': 0.005},
    ],
    'secured_direct': [
        {'threshold_multiplier': 0, 'bonus_rate': 0.0012},
        {'threshold_multiplier': 10, 'bonus_rate': 0.0016},
        {'threshold_multiplier': 25, 'bonus_rate': 0.0025},
        {'threshold_multiplier': 40, 'bonus_rate': 0.0030},
    ],
    'unsecured': [
        {'threshold_multiplier': 0, 'bonus_rate': 0.000},
        {'threshold_multiplier': 3, 'bonus_rate': 0.004},
        {'threshold_multiplier': 6, 'bonus_rate': 0.008},
        {'threshold_multiplier': 9, 'bonus_rate': 0.01},
    ],
}
VINTAGE_SECURED_EFFORT_MATRIX_0_3M = {
    (0, 0):               {(0, 0): 1500, (1, 1): 1800, (2, float('inf')): 2000},
    (1, 1):               {(0, 0): 1800, (1, 1): 2000, (2, float('inf')): 2500},
    (2, 2):               {(0, 0): 2500, (1, 1): 3000, (2, float('inf')): 3500},
    (3, float('inf')):    {(0, 0): 3000, (1, 1): 3500, (2, float('inf')): 4000},
}
VINTAGE_UNSECURED_EFFORT_MATRIX_0_3M = {
    (0, 1):            {(0, 0): 0,   (1,1): 200,  (2, 2): 300, (3, 3): 400, (4, float('inf')): 650},
    (2, 3):            {(0, 0): 200, (1,1): 300,  (2, 2): 400, (3, 3): 650, (4, float('inf')): 750},
    (4, 5):            {(0, 0): 300, (1,1): 400,  (2, 2): 650, (3, 3): 750, (4, float('inf')): 900},
    (6, float('inf')): {(0, 0): 400, (1,1): 650,  (2, 2): 750, (3, 3): 900, (4, float('inf')): 1200},
}

# vintage 4-6 Months
VINTAGE_VOLUME_SLABS_4_6M = {
    'secured': [
        {'threshold_multiplier': 0, 'bonus_rate': 0.000},
        {'threshold_multiplier': 10, 'bonus_rate': 0.0020},
        {'threshold_multiplier': 25, 'bonus_rate': 0.0030},
        {'threshold_multiplier': 40, 'bonus_rate': 0.0040},
        {'threshold_multiplier': 60, 'bonus_rate': 0.0045},
    ],
    'secured_direct': [
        {'threshold_multiplier': 0, 'bonus_rate': 0.0010},
        {'threshold_multiplier': 10, 'bonus_rate': 0.0015},
        {'threshold_multiplier': 25, 'bonus_rate': 0.0025},
        {'threshold_multiplier': 40, 'bonus_rate': 0.0030},
    ],
    'unsecured': [
        {'threshold_multiplier': 0, 'bonus_rate': 0.000},
        {'threshold_multiplier': 4, 'bonus_rate': 0.004},
        {'threshold_multiplier': 12, 'bonus_rate': 0.008},
        {'threshold_multiplier': 16, 'bonus_rate': 0.01},
    ],
}
VINTAGE_SECURED_EFFORT_MATRIX_4_6M = {
    (0, 1):               {(0, 0): 1200, (1, 1): 1500, (2, 2): 1800, (3, float('inf')): 2000},
    (2, 2):               {(0, 0): 1500, (1, 1): 1800, (2, 2): 2000, (3, float('inf')): 2500},
    (3, 3):               {(0, 0): 1800, (1, 1): 2000, (2, 2): 3000, (3, float('inf')): 3500},
    (4, float('inf')):    {(0, 0): 2000, (1, 1): 2500, (2, 2): 3500, (3, float('inf')): 4000},
}
VINTAGE_UNSECURED_EFFORT_MATRIX_4_6M = {
    (0, 3):             {(0, 0): 0,   (1,1): 200,  (2, 2): 300, (3, 4): 400, (5, float('inf')): 500},
    (4, 5):             {(0, 0): 200, (1,1): 250,  (2, 2): 350, (3, 4): 450, (5, float('inf')): 550},
    (6, 7):             {(0, 0): 250, (1,1): 350,  (2, 2): 450, (3, 4): 550, (5, float('inf')): 650},
    (8, 9):             {(0, 0): 350, (1,1): 450,  (2, 2): 550, (3, 4): 650, (5, float('inf')): 750},
    (10, float('inf')): {(0, 0): 450, (1,1): 550,  (2, 2): 650, (3, 4): 750, (5, float('inf')): 1000},
}

# Milestone
MILESTONE_THRESHOLDS = {
    'silver': {'min_secured_files': 2, 'min_volume': 1500000},
    'gold': {'min_secured_files': 3, 'min_volume': 2000000},
    'platinum': {'min_secured_files': 4, 'min_volume': 3000000},
}

MILESTONE_QUALIFYING_MONTHS = {
    'silver': 3,
    'gold': 3,
    'platinum': 3,
} # 5 months evaluation period
MILESTONE_QUALIFYING_MONTHS = {
    'silver': 3,
    'gold': 4,
    'platinum': 4,
} # 6 months evaluation period
MILESTONE_QUALIFYING_MONTHS = {
    'silver': 4,
    'gold': 5,
    'platinum': 5,
} # 7 months evaluation period

MILESTONE_SALARY_HIKE = {
    'silver': {
        'below_25k': 0.08,
        'above_25k': 0.05,
    },
    'gold': {
        'below_25k': 0.15,
        'above_25k': 0.12,
    },
    'platinum': {
        'below_25k': 0.20,
        'above_25k': 0.15,
    },
}

MILESTONE_SATURDAY_OFF = {
    'silver': 1,
    'gold': 2,
    'platinum': 3,
}
#handle the pro-rated payout also
#months required for first evaluation milestone

In [3]:
def get_vintage_category(date_of_joining: datetime.date, current_month_date: datetime.date) -> str:
    april_2025 = datetime.date(2025, 4, 1)
    june_2025 = datetime.date(2025, 6, 30)

    if april_2025 <= current_month_date <= june_2025:
        return '0-3_months'

    if current_month_date > june_2025:
        months_since_april = (current_month_date.year - april_2025.year) * 12 + \
                            (current_month_date.month - april_2025.month)

        if current_month_date.day < april_2025.day:
            months_since_april -= 1

        if 0 <= months_since_april <= 3:
            return '0-3_months'
        elif 4 <= months_since_april <= 6:
            return '4-6_months'
        else:
            return 'BAU'

    months_since_joining = (current_month_date.year - date_of_joining.year) * 12 + \
                           (current_month_date.month - date_of_joining.month)
    if current_month_date.day < date_of_joining.day:
        months_since_joining -= 1

    if 0 <= months_since_joining <= 3:
        return '0-3_months'
    elif 4 <= months_since_joining <= 6:
        return '4-6_months'
    else:
        return 'BAU'

def should_month_be_evaluated(date_of_joining: datetime.date, month_to_evaluate: datetime.date) -> bool:
    vintage_category = get_vintage_category(date_of_joining, month_to_evaluate)

    if vintage_category == '0-3_months' and month_to_evaluate >= datetime.date(2025, 4, 1):
        return True

    if month_to_evaluate < datetime.date(2025, 4, 1):
        if month_to_evaluate.year == date_of_joining.year and \
           month_to_evaluate.month == date_of_joining.month:
            if date_of_joining.day > 10:
                return False

    return True

def get_range_key(value: int, matrix_keys: list[tuple]) -> tuple:
    for r_start, r_end in matrix_keys:
        if r_start <= value <= r_end:
            return (r_start, r_end)
    return None

def calculate_volume_payout(rm_salary: float, volume_achieved: float, volume_type: str, vintage_category: str) -> float:
    payout = 0.0
    current_slabs = VOLUME_PAYOUT_SLABS.get(volume_type, [])

    if vintage_category == '0-3_months' and 'VINTAGE_VOLUME_SLABS_0_3M' in globals():
        current_slabs = globals()['VINTAGE_VOLUME_SLABS_0_3M'].get(volume_type, [])
    elif vintage_category == '4-6_months' and 'VINTAGE_VOLUME_SLABS_4_6M' in globals():
        current_slabs = globals()['VINTAGE_VOLUME_SLABS_4_6M'].get(volume_type, [])

    current_slabs.sort(key=lambda x: x['threshold_multiplier'])
    previous_threshold_volume = 0.0
    remaining_volume = volume_achieved

    for slab in current_slabs:
        slab_upper_volume_threshold = slab['threshold_multiplier'] * rm_salary
        slab_bonus_rate = slab['bonus_rate']
        volume_in_current_slab = min(remaining_volume, slab_upper_volume_threshold - previous_threshold_volume)

        if volume_in_current_slab > 0:
            payout += volume_in_current_slab * slab_bonus_rate
            remaining_volume -= volume_in_current_slab

        previous_threshold_volume = slab_upper_volume_threshold
        if remaining_volume <= 0:
            break

    return payout

def calculate_effort_payout(secured_disbursals: int, secured_logins: int,
                            unsecured_disbursals: int, unsecured_logins: int,
                            vintage_category: str) -> float:
    secured_payout = 0.0
    unsecured_payout = 0.0

    current_secured_matrix = BAU_SECURED_EFFORT_MATRIX
    current_unsecured_matrix = BAU_UNSECURED_EFFORT_MATRIX

    if vintage_category == '0-3_months' and 'VINTAGE_SECURED_EFFORT_MATRIX_0_3M' in globals():
        current_secured_matrix = globals()['VINTAGE_SECURED_EFFORT_MATRIX_0_3M']
        current_unsecured_matrix = globals()['VINTAGE_UNSECURED_EFFORT_MATRIX_0_3M']
    elif vintage_category == '4-6_months' and 'VINTAGE_SECURED_EFFORT_MATRIX_4_6M' in globals():
        current_secured_matrix = globals()['VINTAGE_SECURED_EFFORT_MATRIX_4_6M']
        current_unsecured_matrix = globals()['VINTAGE_UNSECURED_EFFORT_MATRIX_4_6M']

    disb_range_secured = get_range_key(secured_disbursals, list(current_secured_matrix.keys()))
    if disb_range_secured:
        login_range_secured = get_range_key(secured_logins, list(current_secured_matrix[disb_range_secured].keys()))
        if login_range_secured:
            secured_payout = current_secured_matrix[disb_range_secured][login_range_secured]

    disb_range_unsecured = get_range_key(unsecured_disbursals, list(current_unsecured_matrix.keys()))
    if disb_range_unsecured:
        login_range_unsecured = get_range_key(unsecured_logins, list(current_unsecured_matrix[disb_range_unsecured].keys()))
        if login_range_unsecured:
            unsecured_payout = current_unsecured_matrix[disb_range_unsecured][login_range_unsecured]

    total_effort_payout = secured_payout + unsecured_payout
    return total_effort_payout

def check_milestone_achievement(performance_history: list[dict]) -> tuple[str, int]:
    silver_qualifying_months = 0
    gold_qualifying_months = 0
    platinum_qualifying_months = 0

    for month_data in performance_history:
        secured_files = month_data.get('secured_files', 0)
        volume = month_data.get('volume', 0)

        if secured_files >= MILESTONE_THRESHOLDS['silver']['min_secured_files'] and \
           volume >= MILESTONE_THRESHOLDS['silver']['min_volume']:
            silver_qualifying_months += 1

        if secured_files >= MILESTONE_THRESHOLDS['gold']['min_secured_files'] and \
           volume >= MILESTONE_THRESHOLDS['gold']['min_volume']:
            gold_qualifying_months += 1

        if secured_files >= MILESTONE_THRESHOLDS['platinum']['min_secured_files'] and \
           volume >= MILESTONE_THRESHOLDS['platinum']['min_volume']:
            platinum_qualifying_months += 1

    milestone_achieved = None
    saturday_offs = 0

    if platinum_qualifying_months >= MILESTONE_QUALIFYING_MONTHS['platinum']:
        milestone_achieved = 'platinum'
        saturday_offs = MILESTONE_SATURDAY_OFF['platinum']
    elif gold_qualifying_months >= MILESTONE_QUALIFYING_MONTHS['gold']:
        milestone_achieved = 'gold'
        saturday_offs = MILESTONE_SATURDAY_OFF['gold']
    elif silver_qualifying_months >= MILESTONE_QUALIFYING_MONTHS['silver']:
        milestone_achieved = 'silver'
        saturday_offs = MILESTONE_SATURDAY_OFF['silver']

    return milestone_achieved, saturday_offs

def calculate_milestone_hike(rm_salary: float, milestone_achieved: str) -> float:
    if milestone_achieved and milestone_achieved in MILESTONE_SALARY_HIKE:
        hike_rates = MILESTONE_SALARY_HIKE[milestone_achieved]
        if rm_salary < 25000:
            return rm_salary * hike_rates['below_25k']
        else:
            return rm_salary * hike_rates['above_25k']
    return 0.0

def calculate_rm_bonus(
    rm_salary: float,
    date_of_joining: datetime.date,
    current_month_date: datetime.date,
    secured_volume: float,
    secured_direct_volume: float,
    unsecured_volume: float,
    secured_disbursals: int,
    secured_logins: int,
    unsecured_disbursals: int,
    unsecured_logins: int,
    past_performance_history: list[dict]) -> dict:

    vintage_category = get_vintage_category(date_of_joining, current_month_date)
    evaluate_current_month = should_month_be_evaluated(date_of_joining, current_month_date)

    total_volume_payout = 0.0
    total_effort_payout = 0.0
    monthly_payout = 0.0

    if evaluate_current_month:
        total_volume_payout += calculate_volume_payout(rm_salary, secured_volume, 'secured', vintage_category)
        total_volume_payout += calculate_volume_payout(rm_salary, secured_direct_volume, 'secured_direct', vintage_category)
        total_volume_payout += calculate_volume_payout(rm_salary, unsecured_volume, 'unsecured', vintage_category)

        total_effort_payout = calculate_effort_payout(
            secured_disbursals, secured_logins,
            unsecured_disbursals, unsecured_logins,
            vintage_category
        )

        monthly_payout = total_volume_payout + total_effort_payout
    else:
        print(f"Note: Performance for {current_month_date.strftime('%Y-%m')} is NOT evaluated due to joining date rule.")

    milestone_achieved, saturday_offs = check_milestone_achievement(past_performance_history)
    salary_hike_amount = calculate_milestone_hike(rm_salary, milestone_achieved)

    return {
        'monthly_payout': monthly_payout,
        'volume_payout': total_volume_payout,
        'effort_payout': total_effort_payout,
        'milestone_achieved': milestone_achieved,
        'salary_hike_amount': salary_hike_amount,
        'saturday_offs': saturday_offs,
        'vintage_category': vintage_category,
        'current_month_evaluated': evaluate_current_month
    }

### Output Code

In [4]:
if __name__ == "__main__":
    print("--- Example 1: RM in BAU Category (Joined over 6 months ago) ---")
    rm1_salary = 40000
    rm1_joining_date = datetime.date(2023, 8, 10) # Joined Aug 2023, now May 2025 (BAU)
    calculation_month = datetime.date(2025, 5, 31) # Calculating for May 2025

    rm1_secured_volume = 2000000
    rm1_secured_direct_volume = 800000
    rm1_unsecured_volume = 500000
    rm1_secured_disbursals = 3
    rm1_secured_logins = 6
    rm1_unsecured_disbursals = 5
    rm1_unsecured_logins = 12

    rm1_past_performance = [
        {'month_date': datetime.date(2025, 1, 31), 'secured_files': 4, 'volume': 800000},
        {'month_date': datetime.date(2025, 2, 28), 'secured_files': 6, 'volume': 1100000},
        {'month_date': datetime.date(2025, 3, 31), 'secured_files': 7, 'volume': 1300000},
        {'month_date': datetime.date(2025, 4, 30), 'secured_files': 9, 'volume': 1600000},
    ]

    bonus_results_rm1 = calculate_rm_bonus(
        rm_salary=rm1_salary,
        date_of_joining=rm1_joining_date,
        current_month_date=calculation_month,
        secured_volume=rm1_secured_volume,
        secured_direct_volume=rm1_secured_direct_volume,
        unsecured_volume=rm1_unsecured_volume,
        secured_disbursals=rm1_secured_disbursals,
        secured_logins=rm1_secured_logins,
        unsecured_disbursals=rm1_unsecured_disbursals,
        unsecured_logins=rm1_unsecured_logins,
        past_performance_history=rm1_past_performance
    )

    print(f"RM Salary: ₹{rm1_salary:,.2f}")
    print(f"Date of Joining: {rm1_joining_date}")
    print(f"Calculation Month: {calculation_month}")
    print(f"Calculated Vintage Category: {bonus_results_rm1['vintage_category']}")
    print(f"Current Month Evaluated: {bonus_results_rm1['current_month_evaluated']}")
    print(f"Monthly Volume Payout: ₹{bonus_results_rm1['volume_payout']:,.2f}")
    print(f"Monthly Effort Payout: ₹{bonus_results_rm1['effort_payout']:,.2f}")
    print(f"Total Monthly Payout (Effort + Volume): ₹{bonus_results_rm1['monthly_payout']:,.2f}")
    print(f"Milestone Achieved: {bonus_results_rm1['milestone_achieved']}")
    print(f"Salary Hike Amount (from Milestone): ₹{bonus_results_rm1['salary_hike_amount']:,.2f}")
    print(f"Saturday Offs (from Milestone): {bonus_results_rm1['saturday_offs']}")

    # --- Example 2: RM in '0-3_months' Vintage Category ---
    print("\n--- Example 2: RM in '0-3_months' Vintage Category (Joined recently) ---")
    rm2_salary = 20000
    rm2_joining_date = datetime.date(2025, 3, 11) # Joined March 5, 2025, now May 2025 (0-3 months)
    calculation_month_rm2 = datetime.date(2025, 5, 31)

    rm2_secured_volume = 1500000
    rm2_secured_direct_volume = 600000
    rm2_unsecured_volume = 400000
    rm2_secured_disbursals = 4
    rm2_secured_logins = 7
    rm2_unsecured_disbursals = 7
    rm2_unsecured_logins = 15

    rm2_past_performance = [
        {'month_date': datetime.date(2025, 3, 31), 'secured_files': 2, 'volume': 300000},
        {'month_date': datetime.date(2025, 4, 30), 'secured_files': 5, 'volume': 900000},
    ]

    bonus_results_rm2 = calculate_rm_bonus(
        rm_salary=rm2_salary,
        date_of_joining=rm2_joining_date,
        current_month_date=calculation_month_rm2,
        secured_volume=rm2_secured_volume,
        secured_direct_volume=rm2_secured_direct_volume,
        unsecured_volume=rm2_unsecured_volume,
        secured_disbursals=rm2_secured_disbursals,
        secured_logins=rm2_secured_logins,
        unsecured_disbursals=rm2_unsecured_disbursals,
        unsecured_logins=rm2_unsecured_logins,
        past_performance_history=rm2_past_performance
    )

    print(f"RM Salary: ₹{rm2_salary:,.2f}")
    print(f"Date of Joining: {rm2_joining_date}")
    print(f"Calculation Month: {calculation_month_rm2}")
    print(f"Calculated Vintage Category: {bonus_results_rm2['vintage_category']}")
    print(f"Current Month Evaluated: {bonus_results_rm2['current_month_evaluated']}")
    print(f"Monthly Volume Payout: ₹{bonus_results_rm2['volume_payout']:,.2f}")
    print(f"Monthly Effort Payout: ₹{bonus_results_rm2['effort_payout']:,.2f}")
    print(f"Total Monthly Payout (Effort + Volume): ₹{bonus_results_rm2['monthly_payout']:,.2f}")
    print(f"Milestone Achieved: {bonus_results_rm2['milestone_achieved']}")
    print(f"Salary Hike Amount (from Milestone):₹{bonus_results_rm2['salary_hike_amount']:,.2f}")
    print(f"Saturday Offs (from Milestone): {bonus_results_rm2['saturday_offs']}")

    # --- Example 3: RM in '4-6_months' Vintage Category ---
    print("\n--- Example 3: RM in '4-6_months' Vintage Category ---")
    rm3_salary = 35000
    rm3_joining_date = datetime.date(2024, 12, 1) # Joined Dec 1, 2024, now May 2025 (5 months, 4-6 months category)
    calculation_month_rm3 = datetime.date(2025, 5, 31)

    rm3_secured_volume = 1800000
    rm3_secured_direct_volume = 700000
    rm3_unsecured_volume = 450000
    rm3_secured_disbursals = 3
    rm3_secured_logins = 5
    rm3_unsecured_disbursals = 6
    rm3_unsecured_logins = 10

    rm3_past_performance = [
        {'month_date': datetime.date(2025, 1, 31), 'secured_files': 5, 'volume': 900000},
        {'month_date': datetime.date(2025, 2, 28), 'secured_files': 7, 'volume': 1200000},
        {'month_date': datetime.date(2025, 3, 31), 'secured_files': 8, 'volume': 1500000},
        {'month_date': datetime.date(2025, 4, 30), 'secured_files': 10, 'volume': 1800000},
    ]

    bonus_results_rm3 = calculate_rm_bonus(
        rm_salary=rm3_salary,
        date_of_joining=rm3_joining_date,
        current_month_date=calculation_month_rm3,
        secured_volume=rm3_secured_volume,
        secured_direct_volume=rm3_secured_direct_volume,
        unsecured_volume=rm3_unsecured_volume,
        secured_disbursals=rm3_secured_disbursals,
        secured_logins=rm3_secured_logins,
        unsecured_disbursals=rm3_unsecured_disbursals,
        unsecured_logins=rm3_unsecured_logins,
        past_performance_history=rm3_past_performance
    )

    print(f"RM Salary: ₹{rm3_salary:,.2f}")
    print(f"Date of Joining: {rm3_joining_date}")
    print(f"Calculation Month: {calculation_month_rm3}")
    print(f"Calculated Vintage Category: {bonus_results_rm3['vintage_category']}")
    print(f"Current Month Evaluated: {bonus_results_rm3['current_month_evaluated']}")
    print(f"Monthly Volume Payout: ₹{bonus_results_rm3['volume_payout']:,.2f}")
    print(f"Monthly Effort Payout: ₹{bonus_results_rm3['effort_payout']:,.2f}")
    print(f"Total Monthly Payout (Effort + Volume): ₹{bonus_results_rm3['monthly_payout']:,.2f}")
    print(f"Milestone Achieved: {bonus_results_rm3['milestone_achieved']}")
    print(f"Salary Hike Amount (from Milestone): ₹{bonus_results_rm3['salary_hike_amount']:,.2f}")
    print(f"Saturday Offs (from Milestone): {bonus_results_rm3['saturday_offs']}")

--- Example 1: RM in BAU Category (Joined over 6 months ago) ---
RM Salary: ₹40,000.00
Date of Joining: 2023-08-10
Calculation Month: 2025-05-31
Calculated Vintage Category: 0-3_months
Current Month Evaluated: True
Monthly Volume Payout: ₹11,680.00
Monthly Effort Payout: ₹4,900.00
Total Monthly Payout (Effort + Volume): ₹16,580.00
Milestone Achieved: None
Salary Hike Amount (from Milestone): ₹0.00
Saturday Offs (from Milestone): 0

--- Example 2: RM in '0-3_months' Vintage Category (Joined recently) ---
RM Salary: ₹20,000.00
Date of Joining: 2025-03-11
Calculation Month: 2025-05-31
Calculated Vintage Category: 0-3_months
Current Month Evaluated: True
Monthly Volume Payout: ₹6,390.00
Monthly Effort Payout: ₹5,200.00
Total Monthly Payout (Effort + Volume): ₹11,590.00
Milestone Achieved: None
Salary Hike Amount (from Milestone):₹0.00
Saturday Offs (from Milestone): 0

--- Example 3: RM in '4-6_months' Vintage Category ---
RM Salary: ₹35,000.00
Date of Joining: 2024-12-01
Calculation Month

In [8]:
if __name__ == "__main__":
    INPUT_CSV_FILENAME = '/content/RM_Input.csv'
    OUTPUT_CSV_FILENAME = 'rm_bonus_output.csv'

    all_rm_records = []
    rm_historical_data_grouped = defaultdict(list)

    # Expected columns
    expected_columns = [
        'RM_ID', 'RM_Salary', 'Date_of_Joining', 'Month_End_Date',
        'Secured_Volume', 'Secured_Direct_Volume', 'Unsecured_Volume',
        'Secured_Disbursals', 'Secured_Logins', 'Unsecured_Disbursals', 'Unsecured_Logins'
    ]

    def clean_currency_value(value):
        """Clean currency values like '₹ 21,000.00' or '₹ 4,00,043' to float"""
        if not value or value.strip() in ['', '-', ' - ']:
            return 0.0
        # Remove currency symbol, spaces, and commas
        cleaned = value.replace('₹', '').replace(',', '').replace(' ', '')
        try:
            return float(cleaned)
        except ValueError:
            return 0.0

    try:
        with open(INPUT_CSV_FILENAME, mode='r', newline='', encoding='utf-8-sig') as infile:  # utf-8-sig handles BOM
            reader = csv.DictReader(infile)

            # Debug: Print the actual fieldnames detected by csv.DictReader
            print(f"Detected fieldnames: {reader.fieldnames}")

            # Clean fieldnames to remove any BOM or extra characters
            cleaned_fieldnames = [name.strip().replace('\ufeff', '') for name in reader.fieldnames]
            print(f"Cleaned fieldnames: {cleaned_fieldnames}")

            # Check if all expected columns are present after cleaning
            missing_columns = []
            for col in expected_columns:
                if col not in cleaned_fieldnames:
                    missing_columns.append(col)

            if missing_columns:
                print(f"Missing columns: {missing_columns}")
                print("Available columns:", cleaned_fieldnames)
                exit()

            row_count = 0
            for row_num, row in enumerate(reader, start=2):  # start=2 because row 1 is header
                try:
                    # Create a cleaned row dict with proper column names
                    cleaned_row = {}
                    for original_key, value in row.items():
                        cleaned_key = original_key.strip().replace('\ufeff', '')
                        cleaned_row[cleaned_key] = value

                    # Debug: Print first few rows to see the data
                    if row_count < 3:
                        print(f"Row {row_num}: {cleaned_row}")

                    # Check for empty values in critical fields
                    for field in ['RM_ID', 'RM_Salary', 'Date_of_Joining', 'Month_End_Date']:
                        if not cleaned_row[field] or cleaned_row[field].strip() == '':
                            raise ValueError(f"Empty value in required field '{field}' at row {row_num}")

                    processed_row = {
                        'RM_ID': cleaned_row['RM_ID'].strip(),
                        'RM_Salary': clean_currency_value(cleaned_row['RM_Salary']),
                        'Date_of_Joining': datetime.datetime.strptime(cleaned_row['Date_of_Joining'].strip(), '%d-%m-%Y').date(),
                        'Month_End_Date': datetime.datetime.strptime(cleaned_row['Month_End_Date'].strip(), '%d-%m-%Y').date(),
                        'Secured_Volume': clean_currency_value(cleaned_row['Secured_Volume']),
                        'Secured_Direct_Volume': clean_currency_value(cleaned_row['Secured_Direct_Volume']),
                        'Unsecured_Volume': clean_currency_value(cleaned_row['Unsecured_Volume']),
                        'Secured_Disbursals': int(float(cleaned_row['Secured_Disbursals'])) if cleaned_row['Secured_Disbursals'].strip() else 0,
                        'Secured_Logins': int(float(cleaned_row['Secured_Logins'])) if cleaned_row['Secured_Logins'].strip() else 0,
                        'Unsecured_Disbursals': int(float(cleaned_row['Unsecured_Disbursals'])) if cleaned_row['Unsecured_Disbursals'].strip() else 0,
                        'Unsecured_Logins': int(float(cleaned_row['Unsecured_Logins'])) if cleaned_row['Unsecured_Logins'].strip() else 0,
                    }

                    all_rm_records.append(processed_row)
                    rm_historical_data_grouped[processed_row['RM_ID']].append(processed_row)
                    row_count += 1

                except ValueError as ve:
                    print(f"Data conversion error at row {row_num}: {ve}")
                    print(f"Row data: {cleaned_row}")
                    continue
                except Exception as e:
                    print(f"Error processing row {row_num}: {e}")
                    print(f"Row data: {cleaned_row}")
                    continue

            print(f"Successfully processed {row_count} rows")

            # Sort historical data by date for each RM
            for rm_id in rm_historical_data_grouped:
                rm_historical_data_grouped[rm_id].sort(key=lambda x: x['Month_End_Date'])

    except FileNotFoundError:
        print(f"Error: Input CSV file '{INPUT_CSV_FILENAME}' not found. Please create it with the specified format.")
        exit()
    except UnicodeDecodeError:
        print("Error: File encoding issue. Try saving your CSV file as UTF-8 or use a different encoding.")
        exit()
    except Exception as e:
        print(f"Error reading input CSV: {e}")
        import traceback
        traceback.print_exc()
        exit()

    if not all_rm_records:
        print("No valid records found in the CSV file.")
        exit()

    print(f"Processing {len(all_rm_records)} records for bonus calculation...")

    output_data = []
    for record in all_rm_records:
        rm_id = record['RM_ID']
        rm_salary = record['RM_Salary']
        date_of_joining = record['Date_of_Joining']
        current_month_date = record['Month_End_Date']

        try:
            bonus_results = calculate_rm_bonus(
                rm_salary=rm_salary,
                date_of_joining=date_of_joining,
                current_month_date=current_month_date,
                secured_volume=record['Secured_Volume'],
                secured_direct_volume=record['Secured_Direct_Volume'],
                unsecured_volume=record['Unsecured_Volume'],
                secured_disbursals=record['Secured_Disbursals'],
                secured_logins=record['Secured_Logins'],
                unsecured_disbursals=record['Unsecured_Disbursals'],
                unsecured_logins=record['Unsecured_Logins'],
                past_performance_history=[
                    hist_rec for hist_rec in rm_historical_data_grouped.get(rm_id, [])
                    if hist_rec['Month_End_Date'] <= current_month_date
                ]
            )

            output_row = record.copy()
            output_row.update({
                'Monthly_Payout': f"{bonus_results['monthly_payout']:.2f}",
                'Volume_Payout': f"{bonus_results['volume_payout']:.2f}",
                'Effort_Payout': f"{bonus_results['effort_payout']:.2f}",
                'Milestone_Achieved': bonus_results['milestone_achieved'] or '',
                'Salary_Hike_Amount': f"{bonus_results['salary_hike_amount']:.2f}",
                'Saturday_Offs': bonus_results['saturday_offs'],
                'Vintage_Category': bonus_results['vintage_category'],
                'Current_Month_Evaluated': bonus_results['current_month_evaluated']
            })
            output_data.append(output_row)

        except Exception as e:
            print(f"Error calculating bonus for RM {rm_id}: {e}")
            continue

    if output_data:
        original_fieldnames = list(all_rm_records[0].keys())
        output_fieldnames = original_fieldnames + [
            'Monthly_Payout', 'Volume_Payout', 'Effort_Payout', 'Milestone_Achieved',
            'Salary_Hike_Amount', 'Saturday_Offs', 'Vintage_Category', 'Current_Month_Evaluated'
        ]

        # Format the output data
        for row in output_data:
            for key, value in row.items():
                if isinstance(value, datetime.date):
                    row[key] = value.strftime('%Y-%m-%d')
                elif isinstance(value, float):
                    row[key] = f"{value:.2f}"
                elif value is None:
                    row[key] = ''

        try:
            with open(OUTPUT_CSV_FILENAME, mode='w', newline='', encoding='utf-8') as outfile:
                writer = csv.DictWriter(outfile, fieldnames=output_fieldnames)
                writer.writeheader()
                writer.writerows(output_data)
            print(f"\nBonus calculation complete. Results saved to '{OUTPUT_CSV_FILENAME}'")
            print(f"Total records processed: {len(output_data)}")
        except Exception as e:
            print(f"Error writing output CSV: {e}")
    else:
        print("No data processed to write to output CSV.")

Detected fieldnames: ['RM_ID', 'RM_Salary', 'Date_of_Joining', 'Month_End_Date', 'Secured_Volume', 'Secured_Direct_Volume', 'Unsecured_Volume', 'Secured_Disbursals', 'Secured_Logins', 'Unsecured_Disbursals', 'Unsecured_Logins']
Cleaned fieldnames: ['RM_ID', 'RM_Salary', 'Date_of_Joining', 'Month_End_Date', 'Secured_Volume', 'Secured_Direct_Volume', 'Unsecured_Volume', 'Secured_Disbursals', 'Secured_Logins', 'Unsecured_Disbursals', 'Unsecured_Logins']
Row 2: {'RM_ID': 'sf0023', 'RM_Salary': '₹ 21,667.00', 'Date_of_Joining': '28-02-2025', 'Month_End_Date': '31-05-2025', 'Secured_Volume': '₹ 4,80,643', 'Secured_Direct_Volume': '₹ 4,80,643', 'Unsecured_Volume': '₹ 0', 'Secured_Disbursals': '1', 'Secured_Logins': '1', 'Unsecured_Disbursals': '0', 'Unsecured_Logins': '2'}
Row 3: {'RM_ID': 'sf0024', 'RM_Salary': '₹ 20,499.92', 'Date_of_Joining': '01-03-2025', 'Month_End_Date': '31-05-2025', 'Secured_Volume': '₹ 0', 'Secured_Direct_Volume': '₹ 0', 'Unsecured_Volume': '₹ 0', 'Secured_Disbursals